In [1]:
import sys
sys.path.append('../framework')
from NetworkClass import Network
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision
import torch
import torch.nn as nn
import copy


In [2]:
class ReshapeTransform:
    def __init__(self, new_size):
        self.new_size = new_size

    def __call__(self, img):
        return torch.reshape(img, self.new_size)

In [3]:
# Define Constants
batch_size_train = 64
batch_size_test = 1000
n_epochs = 3
learning_rate = 0.01
momentum = 0.5
log_interval = 10
random_seed = 1

In [4]:
# Model Definition
model_dict = {
        "network":{
            'input_layer': {
                "units": 784,
                
                },
            'hidden_layer': [{
                    "units": 400, 
                    "type": "Linear"
                }, 
                {
                    "units": 50, 
                    "activation": "relu",
                    "type": "Linear"

                }],
            'output_layer': {
                "units": 10,
                "activation": "softmax",
                "type": "Linear"
                }
        }
    }

model = Network(model_dict)

softmax


In [5]:
for (layer, param) in enumerate(model.parameters()):
    print("Layer {} , Parameters: {}".format(layer, param.shape))

Layer 0 , Parameters: torch.Size([400, 784])
Layer 1 , Parameters: torch.Size([400])
Layer 2 , Parameters: torch.Size([50, 400])
Layer 3 , Parameters: torch.Size([50])
Layer 4 , Parameters: torch.Size([10, 50])
Layer 5 , Parameters: torch.Size([10])


In [6]:
# Load Datasets

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../data/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,)), ReshapeTransform((-1,))
                             ])),
  batch_size=batch_size_train, shuffle=True)


test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../data/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,)), ReshapeTransform((-1,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [7]:
# Define Loss and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [9]:
def train(mod, optim, epoch):
  mod.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optim.zero_grad()
    output = mod(data)
    loss = criterion(output, target)
    loss.backward()
    optim.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [10]:
def test(mod):
  mod.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = mod(data)
      test_loss += criterion(output, target).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [11]:
test(model)

# Pruning every 3 epochs
for epoch in range(1, n_epochs + 1):
  train(model, optimizer, epoch)
  test(model)
    


../framework/NetworkClass.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.activation_scheme[-1](self.output_layer(x))



Test set: Avg. loss: 0.0023, Accuracy: 1105/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303265
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.301187
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.303478
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.298922
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.302386
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.300650
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.298399
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.303411
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.298145
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.296737
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.296110
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.298870
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.301787
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.296661
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.298602
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.294216
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.293169
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.293176
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.293555
Train Epoch: 1 [12160

Train Epoch: 2 [44160/60000 (74%)]	Loss: 1.822676
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.827644
Train Epoch: 2 [45440/60000 (76%)]	Loss: 1.847715
Train Epoch: 2 [46080/60000 (77%)]	Loss: 1.719860
Train Epoch: 2 [46720/60000 (78%)]	Loss: 1.784843
Train Epoch: 2 [47360/60000 (79%)]	Loss: 1.855408
Train Epoch: 2 [48000/60000 (80%)]	Loss: 1.750753
Train Epoch: 2 [48640/60000 (81%)]	Loss: 1.860564
Train Epoch: 2 [49280/60000 (82%)]	Loss: 1.751453
Train Epoch: 2 [49920/60000 (83%)]	Loss: 1.753484
Train Epoch: 2 [50560/60000 (84%)]	Loss: 1.793297
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.736628
Train Epoch: 2 [51840/60000 (86%)]	Loss: 1.736510
Train Epoch: 2 [52480/60000 (87%)]	Loss: 1.769814
Train Epoch: 2 [53120/60000 (88%)]	Loss: 1.720856
Train Epoch: 2 [53760/60000 (90%)]	Loss: 1.801351
Train Epoch: 2 [54400/60000 (91%)]	Loss: 1.780411
Train Epoch: 2 [55040/60000 (92%)]	Loss: 1.825817
Train Epoch: 2 [55680/60000 (93%)]	Loss: 1.792160
Train Epoch: 2 [56320/60000 (94%)]	Loss: 1.790119


In [74]:
import numpy as np
def prune_neurons(nn_model, pruning_perc=0.1):
    print("Pre-Pruning/n")
    for (layer, param) in enumerate(nn_model.parameters()):
        print("Layer {} , Parameters: {}".format(layer, param.shape))    
        
    # 1 neuron is pruned from each layer based on the minimum L1 norm.
    neurons_to_prune = []
    for p in nn_model.parameters():
            if len(p.data.size()) != 1:
                normed_weights = p.data.abs()
                l1_norm_layer = []
                for neuron_idx in range(normed_weights.shape[0]):
                    l1_norm_layer.append(torch.sum(normed_weights[neuron_idx, :]).item())
                    
                threshold = np.percentile(np.array(l1_norm_layer), pruning_perc * 100)
                prune_idx = np.argwhere(np.array(l1_norm_layer) > threshold).flatten()
           
                neurons_to_prune.append(prune_idx)
                neurons_to_prune.append(prune_idx)                

                
    # Modify the model parameters to update the shape of the network after pruning
    param_list = list(nn_model.parameters())
    
    
    for i, neuron_idx in enumerate(neurons_to_prune):
        idx_weights = param_list[i]
        if i < len(param_list) - 2:
            print(idx_weights.shape)
            y = idx_weights[neuron_idx]
            if i > 1 and len(idx_weights.shape)> 1:
                y = y[:, neurons_to_prune[i-1]]
        elif i > 1 and len(idx_weights.shape) > 1:
            y = idx_weights[:, neurons_to_prune[i-1]]
            
        else:
            y = idx_weights
            
        idx_weights.data = y
#     for i, neuron_indexes in enumerate(neurons_to_prune):
#         index_shift = 0
#         for neuron_idx in neuron_indexes:
#             idx_weights = param_list[i]
#             if i < len(param_list) - 2:
#                 y = torch.cat((idx_weights[0:neuron_idx + index_shift], idx_weights[neuron_idx+1 + index_shift:]))
#                 if i > 1 and len(idx_weights.shape) > 1:

#                     y = torch.cat((y[:, 0:neuron_idx+index_shift-1], y[:, neuron_idx+index_shift:]), axis=1)
#                 index_shift = index_shift - 1

#             elif i > 1 and len(idx_weights.shape) > 1:
#                 y = torch.cat((idx_weights[:, 0:neuron_idx-1+index_shift], idx_weights[:, neuron_idx+index_shift:]), axis=1)
#                 index_shift = index_shift - 1

#             else:
#                 y = idx_weights
#             idx_weights.data = y  
            
       
    print("Post Pruning /n")
    for (layer, param) in enumerate(nn_model.parameters()):
        print("Layer {} , Parameters: {}".format(layer, param.shape))   
        
    return nn_model

In [75]:
def prune_train(model, optimizer):
    new_model = copy.deepcopy(model)
    new_optimizer = copy.deepcopy(optimizer)
    
    # Get new model after pruning
    new_model = prune_neurons(new_model)
    new_model_dict = copy.deepcopy(model_dict)
    
    
    for i, layer in enumerate(model_dict['network']['hidden_layer']):
        if i == 0:

            new_model_dict['network']['hidden_layer'][i]['units'] = new_model.state_dict()['input_layer.weight'].shape[0]
        else:
            new_model_dict['network']['hidden_layer'][i]['units'] = new_model.state_dict()['hidden_layers.{}.weight'.format(i-1)].shape[0]

    # Create new model based on updated network definition
    updated_model = Network(new_model_dict)
    
    # Load previously trained parameters into the new model
    updated_model.load_state_dict(new_model.state_dict())
    
    # Update Optimizers and set state from the previous optimizer
    criterion = nn.CrossEntropyLoss()
    updated_optimizer = torch.optim.SGD(updated_model.parameters(), lr=learning_rate)
    updated_optimizer.load_state_dict(new_optimizer.state_dict())
    
    # Train and test the model
    for epoch in range(1, n_epochs + 1):
      train(updated_model, updated_optimizer, epoch)
      test(updated_model)
        
    return updated_model, updated_optimizer

In [76]:
# Perform Pruning 5 times in succession
for i in range(5):
    model, optimizer = prune_train(model, optimizer)


Pre-Pruning/n
Layer 0 , Parameters: torch.Size([400, 784])
Layer 1 , Parameters: torch.Size([400])
Layer 2 , Parameters: torch.Size([50, 400])
Layer 3 , Parameters: torch.Size([50])
Layer 4 , Parameters: torch.Size([10, 50])
Layer 5 , Parameters: torch.Size([10])
torch.Size([400, 784])
torch.Size([400])
torch.Size([50, 400])
torch.Size([50])
Post Pruning /n
Layer 0 , Parameters: torch.Size([360, 784])
Layer 1 , Parameters: torch.Size([360])
Layer 2 , Parameters: torch.Size([45, 360])
Layer 3 , Parameters: torch.Size([45])
Layer 4 , Parameters: torch.Size([10, 45])
Layer 5 , Parameters: torch.Size([10])
softmax
Train Epoch: 1 [0/60000 (0%)]	Loss: 1.627149
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.696276
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.584092
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.644291
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.650313
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.592292
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.600010
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.604287
Tr

Train Epoch: 2 [37120/60000 (62%)]	Loss: 1.538404
Train Epoch: 2 [37760/60000 (63%)]	Loss: 1.598477
Train Epoch: 2 [38400/60000 (64%)]	Loss: 1.612613
Train Epoch: 2 [39040/60000 (65%)]	Loss: 1.567842
Train Epoch: 2 [39680/60000 (66%)]	Loss: 1.666759
Train Epoch: 2 [40320/60000 (67%)]	Loss: 1.641897
Train Epoch: 2 [40960/60000 (68%)]	Loss: 1.576297
Train Epoch: 2 [41600/60000 (69%)]	Loss: 1.558138
Train Epoch: 2 [42240/60000 (70%)]	Loss: 1.611174
Train Epoch: 2 [42880/60000 (71%)]	Loss: 1.566072
Train Epoch: 2 [43520/60000 (72%)]	Loss: 1.590023
Train Epoch: 2 [44160/60000 (74%)]	Loss: 1.618600
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.560778
Train Epoch: 2 [45440/60000 (76%)]	Loss: 1.515061
Train Epoch: 2 [46080/60000 (77%)]	Loss: 1.578439
Train Epoch: 2 [46720/60000 (78%)]	Loss: 1.563235
Train Epoch: 2 [47360/60000 (79%)]	Loss: 1.584521
Train Epoch: 2 [48000/60000 (80%)]	Loss: 1.580007
Train Epoch: 2 [48640/60000 (81%)]	Loss: 1.560518
Train Epoch: 2 [49280/60000 (82%)]	Loss: 1.623059


Train Epoch: 1 [13440/60000 (22%)]	Loss: 1.630524
Train Epoch: 1 [14080/60000 (23%)]	Loss: 1.588751
Train Epoch: 1 [14720/60000 (25%)]	Loss: 1.575962
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.512063
Train Epoch: 1 [16000/60000 (27%)]	Loss: 1.612209
Train Epoch: 1 [16640/60000 (28%)]	Loss: 1.623717
Train Epoch: 1 [17280/60000 (29%)]	Loss: 1.529647
Train Epoch: 1 [17920/60000 (30%)]	Loss: 1.563693
Train Epoch: 1 [18560/60000 (31%)]	Loss: 1.560568
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.571598
Train Epoch: 1 [19840/60000 (33%)]	Loss: 1.625340
Train Epoch: 1 [20480/60000 (34%)]	Loss: 1.585507
Train Epoch: 1 [21120/60000 (35%)]	Loss: 1.559599
Train Epoch: 1 [21760/60000 (36%)]	Loss: 1.583847
Train Epoch: 1 [22400/60000 (37%)]	Loss: 1.563200
Train Epoch: 1 [23040/60000 (38%)]	Loss: 1.542488
Train Epoch: 1 [23680/60000 (39%)]	Loss: 1.525010
Train Epoch: 1 [24320/60000 (41%)]	Loss: 1.534806
Train Epoch: 1 [24960/60000 (42%)]	Loss: 1.605911
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.531267


Train Epoch: 2 [58880/60000 (98%)]	Loss: 1.553377
Train Epoch: 2 [59520/60000 (99%)]	Loss: 1.587857

Test set: Avg. loss: 0.0016, Accuracy: 9204/10000 (92%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.570523
Train Epoch: 3 [640/60000 (1%)]	Loss: 1.501665
Train Epoch: 3 [1280/60000 (2%)]	Loss: 1.592877
Train Epoch: 3 [1920/60000 (3%)]	Loss: 1.548163
Train Epoch: 3 [2560/60000 (4%)]	Loss: 1.527000
Train Epoch: 3 [3200/60000 (5%)]	Loss: 1.581389
Train Epoch: 3 [3840/60000 (6%)]	Loss: 1.583590
Train Epoch: 3 [4480/60000 (7%)]	Loss: 1.538975
Train Epoch: 3 [5120/60000 (9%)]	Loss: 1.555700
Train Epoch: 3 [5760/60000 (10%)]	Loss: 1.511120
Train Epoch: 3 [6400/60000 (11%)]	Loss: 1.577709
Train Epoch: 3 [7040/60000 (12%)]	Loss: 1.590466
Train Epoch: 3 [7680/60000 (13%)]	Loss: 1.509999
Train Epoch: 3 [8320/60000 (14%)]	Loss: 1.679116
Train Epoch: 3 [8960/60000 (15%)]	Loss: 1.548100
Train Epoch: 3 [9600/60000 (16%)]	Loss: 1.582197
Train Epoch: 3 [10240/60000 (17%)]	Loss: 1.533246
Train Epoch: 3 [10880

Train Epoch: 1 [35840/60000 (60%)]	Loss: 1.536058
Train Epoch: 1 [36480/60000 (61%)]	Loss: 1.535447
Train Epoch: 1 [37120/60000 (62%)]	Loss: 1.551520
Train Epoch: 1 [37760/60000 (63%)]	Loss: 1.573225
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.541418
Train Epoch: 1 [39040/60000 (65%)]	Loss: 1.545272
Train Epoch: 1 [39680/60000 (66%)]	Loss: 1.510252
Train Epoch: 1 [40320/60000 (67%)]	Loss: 1.551261
Train Epoch: 1 [40960/60000 (68%)]	Loss: 1.540670
Train Epoch: 1 [41600/60000 (69%)]	Loss: 1.556045
Train Epoch: 1 [42240/60000 (70%)]	Loss: 1.528964
Train Epoch: 1 [42880/60000 (71%)]	Loss: 1.514779
Train Epoch: 1 [43520/60000 (72%)]	Loss: 1.561605
Train Epoch: 1 [44160/60000 (74%)]	Loss: 1.526430
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.542726
Train Epoch: 1 [45440/60000 (76%)]	Loss: 1.604077
Train Epoch: 1 [46080/60000 (77%)]	Loss: 1.578504
Train Epoch: 1 [46720/60000 (78%)]	Loss: 1.536518
Train Epoch: 1 [47360/60000 (79%)]	Loss: 1.581894
Train Epoch: 1 [48000/60000 (80%)]	Loss: 1.568531


Train Epoch: 3 [20480/60000 (34%)]	Loss: 1.544302
Train Epoch: 3 [21120/60000 (35%)]	Loss: 1.557665
Train Epoch: 3 [21760/60000 (36%)]	Loss: 1.569470
Train Epoch: 3 [22400/60000 (37%)]	Loss: 1.518181
Train Epoch: 3 [23040/60000 (38%)]	Loss: 1.582706
Train Epoch: 3 [23680/60000 (39%)]	Loss: 1.569720
Train Epoch: 3 [24320/60000 (41%)]	Loss: 1.531758
Train Epoch: 3 [24960/60000 (42%)]	Loss: 1.553185
Train Epoch: 3 [25600/60000 (43%)]	Loss: 1.544106
Train Epoch: 3 [26240/60000 (44%)]	Loss: 1.523059
Train Epoch: 3 [26880/60000 (45%)]	Loss: 1.532205
Train Epoch: 3 [27520/60000 (46%)]	Loss: 1.539677
Train Epoch: 3 [28160/60000 (47%)]	Loss: 1.521640
Train Epoch: 3 [28800/60000 (48%)]	Loss: 1.539106
Train Epoch: 3 [29440/60000 (49%)]	Loss: 1.547082
Train Epoch: 3 [30080/60000 (50%)]	Loss: 1.493491
Train Epoch: 3 [30720/60000 (51%)]	Loss: 1.548985
Train Epoch: 3 [31360/60000 (52%)]	Loss: 1.539915
Train Epoch: 3 [32000/60000 (53%)]	Loss: 1.523911
Train Epoch: 3 [32640/60000 (54%)]	Loss: 1.560562


Train Epoch: 1 [56960/60000 (95%)]	Loss: 1.523591
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.495860
Train Epoch: 1 [58240/60000 (97%)]	Loss: 1.519038
Train Epoch: 1 [58880/60000 (98%)]	Loss: 1.501283
Train Epoch: 1 [59520/60000 (99%)]	Loss: 1.547519

Test set: Avg. loss: 0.0015, Accuracy: 9303/10000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.527869
Train Epoch: 2 [640/60000 (1%)]	Loss: 1.563751
Train Epoch: 2 [1280/60000 (2%)]	Loss: 1.531582
Train Epoch: 2 [1920/60000 (3%)]	Loss: 1.533167
Train Epoch: 2 [2560/60000 (4%)]	Loss: 1.549517
Train Epoch: 2 [3200/60000 (5%)]	Loss: 1.506093
Train Epoch: 2 [3840/60000 (6%)]	Loss: 1.563376
Train Epoch: 2 [4480/60000 (7%)]	Loss: 1.580098
Train Epoch: 2 [5120/60000 (9%)]	Loss: 1.538028
Train Epoch: 2 [5760/60000 (10%)]	Loss: 1.543878
Train Epoch: 2 [6400/60000 (11%)]	Loss: 1.525607
Train Epoch: 2 [7040/60000 (12%)]	Loss: 1.511425
Train Epoch: 2 [7680/60000 (13%)]	Loss: 1.558866
Train Epoch: 2 [8320/60000 (14%)]	Loss: 1.548605
Train Epoch: 2 [896

KeyboardInterrupt: 